In [ ]:
import pandas as pd
import sqlite3

# Load and format the data to be compatible with SQL database
stock_data = pd.read_csv("/content/StockMarket_Description.csv")
stock_data.rename(columns={
    'Symbol': 'symbol',
    'Security': 'security',
    'GICS Sector': 'gics_sector',
    'GICS Sub-Industry': 'gics_sub_industry',
    'Headquarters Location': 'headquarters_location',
    'Date added': 'date_added',
    'CIK': 'cik',
    'Founded': 'founded'
}, inplace=True)


# Save to SQLite
conn = sqlite3.connect("stockMarket_Description.db")
stock_data.to_sql("stockmarket_description", conn, if_exists="replace", index=False)
conn.close()

# Make sure data in founded colun
print(stock_data['founded'].dtype)
print(stock_data['founded'].unique())
# Clear and Format the founded year
#stock_data.head()

In [19]:
import sqlite3
import spacy

nlp = spacy.load("en_core_web_sm")

# Helper function to query the SQLite database
def query_db(query, params=()):
    conn = sqlite3.connect("stockMarket_Description.db")
    cursor = conn.cursor()
    cursor.execute(query, params)
    result = cursor.fetchall()
    conn.close()
    return result

# Extract 4-digit year from message using spaCy
def extract_year(text):
    for ent in nlp(text).ents:
        if ent.label_ in {"DATE", "CARDINAL"}:
            digits = ''.join(filter(str.isdigit, ent.text))
            if len(digits) == 4:
                return int(digits)
    return None

# Extract US state or location using spaCy
def extract_state(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "GPE":
            return ent.text
    return None

# Handle query for companies by HQ and sector
def handle_hq_sector(msg):
    parts = msg.lower().split("headquartered in")[-1].split(" and ")
    if len(parts) < 2:
        return "Try: 'Companies headquartered in Texas and Energy sector'"
    hq, sector = parts[0].strip(), parts[1].replace("sector", "").strip()
    result = query_db(
        "SELECT security FROM stockmarket_description WHERE headquarters_location LIKE ? AND gics_sector LIKE ?",
        (f"%{hq}%", f"%{sector}%")
    )
    return f"Matches: {', '.join(r[0] for r in result)}" if result else "No matches found."

# Handle query for companies founded before a given year
def handle_founded_before(msg):
    year = extract_year(msg)
    if not year:
        return "Couldn't extract a valid year."
    result = query_db("SELECT security, founded FROM stockmarket_description")

    matches = []
    for name, founded in result:
        try:
            if isinstance(founded, (int, float)) and int(founded) < year:
                matches.append(name)
            elif isinstance(founded, str) and founded.isdigit() and int(founded) < year:
                matches.append(name)
        except:
            continue

    return f"Founded before {year}: {', '.join(matches)}" if matches else "No companies found."

# Handle query for companies founded after a given year
def handle_founded_after(msg):
    year = extract_year(msg)
    if not year:
        return "Couldn't extract a valid year."
    result = query_db("SELECT security, founded FROM stockmarket_description")

    matches = []
    for name, founded in result:
        try:
            if isinstance(founded, (int, float)) and int(founded) > year:
                matches.append(name)
            elif isinstance(founded, str) and founded.isdigit() and int(founded) > year:
                matches.append(name)
        except:
            continue

    return f"Founded after {year}: {', '.join(matches)}" if matches else "No companies found."

# Handle query for companies by state
def handle_by_state(msg):
    state = extract_state(msg)
    if not state:
        return "Could not determine the state."
    result = query_db(
        "SELECT security FROM stockmarket_description WHERE headquarters_location LIKE ?",
        (f"%, {state}",)
    )
    return f"Companies in {state}: {', '.join(r[0] for r in result)}" if result else f"No companies found in {state}."

# Handle query for company count per sector
def handle_sector_count():
    result = query_db("SELECT gics_sector, COUNT(*) FROM stockmarket_description GROUP BY gics_sector ORDER BY COUNT(*) DESC")
    return "Companies per sector:\n" + "\n".join(f"{sector}: {count}" for sector, count in result)

# Handle query to list all companies
def handle_list_all():
    result = query_db("SELECT security FROM stockmarket_description")
    return ", ".join(r[0] for r in result)

# Handle query for company details
def handle_company_lookup(msg):
    company = msg.lower().replace("tell me about", "").replace("?", "").strip()
    result = query_db("SELECT * FROM stockmarket_description WHERE LOWER(security) LIKE ?", (f"%{company}%",))
    return str(result[0]) if result else "That company is not in the S&P 500 list."

In [20]:
import re

def respond(msg):
    msg_lower = msg.lower()

    # Updated regex patterns for broader natural phrasing
    hq_sector_pattern = r"headquartered in(.*?)and(.*?)sector"
    founded_before_pattern = r"founded.*before.*?(\d{4})"
    founded_after_pattern = r"founded.*(after|in or after).*?(\d{4})"
    hq_state_pattern = r"headquartered in (.*)"
    sector_count_pattern = r"how many companies are in each sector"
    list_all_pattern = r"list all companies"
    company_lookup_pattern = r"tell me about (.*)"

    # Check for matches and call corresponding functions
    if re.search(hq_sector_pattern, msg_lower):
        return handle_hq_sector(msg)
    elif re.search(founded_before_pattern, msg_lower):
        return handle_founded_before(msg)
    elif re.search(founded_after_pattern, msg_lower):
        return handle_founded_after(msg)
    elif re.search(hq_state_pattern, msg_lower):
        return handle_by_state(msg)
    elif re.search(sector_count_pattern, msg_lower):
        return handle_sector_count()
    elif re.search(list_all_pattern, msg_lower):
        return handle_list_all()
    elif re.search(company_lookup_pattern, msg_lower):
        return handle_company_lookup(msg)
    else:
        return "Try asking about HQ location, founding year, sectors, or a specific company."

In [22]:
def chat():
    print("Ask me about the S&P 500!")
    print("You can ask things like:")
    print("- 'List companies headquartered in New York and Financials sector'")
    print("- 'Which companies were founded before 1950'")
    print("- 'Which companies were founded after 2000'")
    print("- 'Show me companies headquartered in California'")
    print("- 'How many companies are in each sector'")
    print("- 'List all companies'")
    print("- 'Tell me about Apple'\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit", "bye"}:
            print("Goodbye!")
            break
        print("Bot:", respond(user_input))


chat()

Ask me about the S&P 500!
You can ask things like:
- 'List companies headquartered in New York and Financials sector'
- 'Which companies were founded before 1950'
- 'Which companies were founded after 2000'
- 'Show me companies headquartered in California'
- 'How many companies are in each sector'
- 'List all companies'
- 'Tell me about Apple'

You: List companies headquartered in Massachusetts and Financial sector
Bot: Matches: State Street Corporation
You: Which companies were founded before 1950
Bot: Founded before 1950: Abbott Laboratories, Archer Daniels Midland, Automatic Data Processing, Ameren, American Electric Power, American International Group, Assurant, Arthur J. Gallagher & Co., Allstate, Allegion, Ametek, Ameriprise Financial, A. O. Smith, Air Products, Amphenol, Atmos Energy, Avery Dennison, American Water Works, American Express, Boeing, Ball Corporation, Baxter International, Becton Dickinson, Franklin Resources, Brown–Forman, Bunge Global, BNY Mellon, Berkshire Hatha